# Insect Classification Pipeline with Bplusplus

This notebook demonstrates a complete pipeline for detecting and classifying insects in videos using the Bplusplus library.

## What you'll accomplish:
1. **Collect** insect images from GBIF database
2. **Prepare** data by detecting and cropping insects from raw images  
3. **Train** a classification model for family, genus, and species identification
4. **Test** the trained model's performance
5. **Run inference** on videos to detect, classify, and track insects over time

## How it works:
- Uses a **two-stage approach**: pre-trained detection + custom classification
- Detection stage: Locates insects in images/video frames
- Classification stage: Identifies insects at three taxonomic levels (family → genus → species)
- Tracking stage: Follows insects over time and aggregates predictions per individual

By the end, you'll have a system that can process videos and output the most likely classification for each tracked insect.



## Setup: Create Virtual Environment (Recommended)

Create an isolated environment to avoid package conflicts:

```bash
python3 -m venv bplusplus_env
source bplusplus_env/bin/activate
```

## Setup: Install Required Packages

In [ ]:
! pip install bplusplus

## Import required packages

## Set directories

In [ ]:
import bplusplus
from typing import Any
from pathlib import Path
import requests
from tqdm import tqdm

In [ ]:
MAIN_DIR = Path("./")

GBIF_DATA_DIR = MAIN_DIR / "GBIF_data"
PREPARED_DATA_DIR = MAIN_DIR / "prepared_data"
TRAINED_MODEL_DIR = MAIN_DIR / "trained_model"

## Step 1: Collect Insect Images from GBIF

We download images from the GBIF (Global Biodiversity Information Facility) database for our target species.

**Important notes:**
- Download more images than needed - many will be filtered out during preparation
- Internet connection may be unstable - monitor progress and resume if needed
- Check the `GBIF_DATA_DIR` folder to track downloaded files

In [ ]:
names = [
        "Coccinella septempunctata", "Apis mellifera", "Bombus lapidarius", "Bombus terrestris",
        "Eupeodes corollae", "Episyrphus balteatus", "Aglais urticae", "Vespula vulgaris",
        "Eristalis tenax"
    ]



search: dict[str, Any] = {
    "scientificName": names
}

In [ ]:
bplusplus.collect(
    group_by_key=bplusplus.Group.scientificName,
    search_parameters=search, 
    images_per_group=400,
    output_directory=GBIF_DATA_DIR,
    num_threads=3
)


## Step 2: Prepare Data for Training

This step uses a pre-trained vision model to:
1. **Detect** insects in the raw GBIF images at confidence threshold `conf`.
2. **Crop** each detected insect to focus on the subject
3. **Optionally blur** images to reduce noise (set `blur` as fraction of image size, e.g. 0.01 = 1%)
4. **Resize** images to a consistent size for training `img_size`

**What to expect:**
- Many images will be rejected (low success rate is normal)
- Only clear, well-detected insects proceed to training
- This filtering ensures high-quality training data
- Supported formats: JPG, JPEG, and PNG

In [ ]:
bplusplus.prepare(
    input_directory=GBIF_DATA_DIR,
    output_directory=PREPARED_DATA_DIR,
    img_size=60,  # resized image size
    conf=0.6,     # confidence threshold for insect detection weights
    valid=0.1,    # split into validation set (0 - 1)
    blur=None,    # gaussian blur as fraction of image size (0-1), e.g. 0.01 = 1%
)

## Step 3: Train the Classification Model

Train a ResNet-based neural network that classifies insects at three taxonomic levels:
- **Family** (e.g., Coccinellidae for ladybugs)
- **Genus** (e.g., Coccinella)  
- **Species** (e.g., Coccinella septempunctata)

The model learns hierarchical relationships between these classification levels. 

In [ ]:
import torchvision.transforms as transforms


bplusplus.train(
    batch_size=4,
    epochs=9,
    patience=3,
    img_size=60,
    num_workers=0, #force single process, most stable
    data_dir=PREPARED_DATA_DIR,
    output_dir=TRAINED_MODEL_DIR,
    species_list=names,
    train_transforms= transforms.Compose([ #for custom transforms below is default
       transforms.RandomResizedCrop(60),
       transforms.RandomHorizontalFlip(),
       transforms.RandomVerticalFlip(),
       transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
       transforms.RandomPerspective(distortion_scale=0.2),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    backbone="resnet50" #for custom backbone below is default (choose between resnet18, resnet50 and resnet101)
)


## Step 4: Validate Model

Evaluates a trained hierarchical model on a held-out dataset, calculating precision, recall, and F1-score at all taxonomic levels (Family, Genus, Species).
Directory structure required:

```
validation_dir/
├── Apis mellifera/
│   ├── image1.jpg
│   └── image2.jpg
├── Bombus terrestris/
│   └── image1.jpg
└── ...
```

In [ ]:
RESNET_MULTITASK_WEIGHTS = TRAINED_MODEL_DIR / "best_multitask.pt"

In [ ]:
results = bplusplus.validate(
    species_list=names,
    validation_dir="./valid",
    hierarchical_weights=RESNET_MULTITASK_WEIGHTS,
    img_size=60,           # Must match training
    batch_size=32,          # Adjust based on GPU memory
    backbone="resnet50"     # Must match training
)


## Step 5: Run Video Inference

Runs motion-based insect detection and hierarchical classification on video files. Detects moving insects using background subtraction (GMM), tracks them across frames, classifies each detection, and aggregates predictions per track.

**Output files generated:**
- `{video}_annotated.mp4` - Video with detection boxes and track paths (if `save_video=True`)
- `{video}_debug.mp4` - Side-by-side view with GMM motion mask (if `save_video=True`)
- `{video}_results.csv` - Aggregated results per track
- `{video}_detections.csv` - Frame-by-frame detections

**Tip:** Set `save_video=False` to skip video rendering and only generate CSV output (faster processing).

In [ ]:
results = bplusplus.inference(
    species_list=names,
    hierarchical_model_path=RESNET_MULTITASK_WEIGHTS,
    video_path="./10.mp4",
    output_dir="./output",
    fps=None,              # None = all frames
    backbone="resnet50",   # Must match training
    save_video=True,       # Set to False to skip video rendering (only CSV output)
    img_size=60,           # Must match training
)

print(f"Detected {results['tracks']} tracks ({results['confirmed_tracks']} confirmed)")

### Custom Detection Configuration

The inference uses motion-based detection with configurable parameters for filtering detections. You can customize these by providing a YAML or JSON config file.

Download a template config from:  https://github.com/Tvenver/Bplusplus/releases/download/weights/detection_config.yaml

```python
results = bplusplus.inference(
    ...,
    config="detection_config.yaml"
)
```

| Parameter | Default | Description |
|-----------|---------|-------------|
| **Cohesiveness** | | *Filters scattered motion (plants) vs compact motion (insects)* |
| `min_largest_blob_ratio` | 0.80 | Min ratio of largest blob to total motion |
| `max_num_blobs` | 5 | Max separate blobs allowed in detection |
| **Shape** | | *Filters by contour properties* |
| `min_area` | 200 | Min detection area (px²) |
| `max_area` | 40000 | Max detection area (px²) |
| `min_density` | 3.0 | Min area/perimeter ratio |
| `min_solidity` | 0.55 | Min convex hull fill ratio |
| **Tracking** | | *Controls track behavior* |
| `min_displacement` | 50 | Min net movement for confirmation (px) |
| `min_path_points` | 10 | Min points before path analysis |
| `max_frame_jump` | 100 | Max jump between frames (px) |
| `lost_track_seconds` | 1.5 | How long to remember lost tracks (s) |
| **Path Topology** | | *Confirms insect-like movement patterns* |
| `max_revisit_ratio` | 0.30 | Max ratio of revisited positions |
| `min_progression_ratio` | 0.70 | Min forward progression |
| `max_directional_variance` | 0.90 | Max heading variance |

